In [ ]:
import re
import numpy as np

with open('input.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
disk_map = list(map(int, list(lines[0])))

# Files in disk map as tuples (offset, file_id, length).
blocks = []
offset = 0
file_id = 0

for i in range(len(disk_map)):
  if i % 2 == 0:
    blocks.append((offset, file_id, disk_map[i]))
    file_id += 1
  offset += disk_map[i]

def show_layout(blocks):
  vis = ''
  x = 0
  for (offset, file_id, length) in blocks:
    while x < offset:
      vis += '.'
      x += 1
    for i in range(length):
      vis += f'{file_id}'
      x += 1
  return vis

def defrag(blocks):
  result = []
  i, j = 0, len(blocks) - 1
  write_offset = 0

  while i <= j:
    # Find out if there's already a block starting at write_offset, or if we need to fill some free space.
    (head_offset, head_file_id, head_length) = blocks[i]
    (tail_offset, tail_file_id, tail_length) = blocks[j]

    if head_offset == write_offset:
      result.append(blocks[i])
      write_offset += head_length
      i += 1
    else:
      # Reallocate from the tail to fill the free space:
      free_space = head_offset - write_offset
      if tail_length <= free_space:
        result.append((write_offset, tail_file_id, tail_length))
        write_offset += tail_length
        j -= 1
      else:
        blocks = blocks[:j] + [(tail_offset, tail_file_id, tail_length - free_space)] + blocks[(j + 1):]
        result.append((write_offset, tail_file_id, free_space))
        write_offset += free_space

  return result

blocks_defragged = defrag(blocks)

In [ ]:
def checksum(blocks):
  last_elem = blocks[-1]
  eof_offset = last_elem[0] + last_elem[2]

  result = 0
  for (b_offset, b_file_id, b_length) in blocks:
    for i in range(b_length):
      result += (b_offset + i) * b_file_id
  return result

checksum(blocks_defragged)